In [23]:
cd img

[Errno 2] No such file or directory: 'img'
/Users/apple/img


In [2]:
%matplotlib inline 
from selenium import webdriver
import time,re,json,numpy as np
import pandas as pd
from collections import defaultdict,Counter
import matplotlib.pyplot as plt

/Users/apple/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:1041: UserWarning: Duplicate key in file "/Users/apple/.matplotlib/matplotlibrc", line #2
  (fname, cnt))


In [3]:
url = "http://www.imdb.com/list/ls061683439/"
with open('filmfare.json',encoding="utf-8") as f:
    datatbl = json.load(f)
driver = webdriver.Chrome(datatbl['data']['chromedriver'])
driver.get(url)

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'lxml')
lstelem = soup.findAll("div", attrs={"class" : 'info'})
Movielist = []
for lst in lstelem:
    Movielist.append((lst.find('b').contents[0]).text)
print("First 10 Movies in the list")
Movielist[0:10]

First 10 Movies in the list


['Bajirao Mastani',
 'Queen',
 'Bhaag Milkha Bhaag',
 'Barfi!',
 'Zindagi Na Milegi Dobara',
 'Dabangg',
 '3 Idiots',
 'Jodhaa Akbar',
 'Like Stars on Earth',
 'Rang De Basanti']

In [5]:
def ExtractText(Xpath):
    textlist=[]
    if(Xpath=='Movies_Director_Xpath'):
        for item in range(1,123,2):
            textlist.append(driver.find_element_by_xpath(datatbl['data'][Xpath]+'[%d]'%item).text)
    else:
        [textlist.append(item.text) for item in driver.find_elements_by_xpath(datatbl['data'][Xpath])]
    return textlist

In [6]:
#Extracting Data from Web
Movies_Votes,Movies_Name,Movies_Ratings,Movies_RunTime=[[] for i in range(4)]
datarepo = [[]]*5
Xpath_list = ['Movies_Name_Xpath','Movies_Rate_Xpath','Movies_Runtime_Xpath','Movies_Votes_Xpath',
              'Movies_Director_Xpath']

for i in range(5):
    if(i==3):
        driver.find_element_by_xpath(datatbl['data']['listview']).click()
    if(i==4):
        driver.find_element_by_xpath(datatbl['data']['detailview']).click()
    datarepo[i] = ExtractText(Xpath_list[i])

driver.quit()

In [7]:
# Movie Name List & Ratings
print(datarepo[0][:5])
print(datarepo[1][:5])
print(datarepo[3][:5])
print(datarepo[4][:5])
print("")
print(datarepo[3][:5])

['Bajirao Mastani', 'Queen', 'Bhaag Milkha Bhaag', 'Barfi!', 'Zindagi Na Milegi Dobara']
['7.2', '8.3', '8.3', '8.1', '8.1']
['22,156', '46,678', '46,431', '59,944', '48,927']
['Director: Sanjay Leela Bhansali', 'Director: Vikas Bahl', 'Director: Rakeysh Omprakash Mehra', 'Director: Anurag Basu', 'Director: Zoya Akhtar']

['22,156', '46,678', '46,431', '59,944', '48,927']


In [8]:
# Result in a Python Dictionary
Years=range(2015,1954,-1)
result = defaultdict(dict)
for i in range(0,len(datarepo[0])):
    result[i]['Movie Name']= datarepo[0][i]
    result[i]['Year']= Years[i]
    result[i]['Rating']= datarepo[1][i]
    result[i]['Votes']= datarepo[3][i]
    result[i]['RunTime']= datarepo[2][i]
    result[i]['Genre']= datatbl['data']['Genre'][i]
    result[i]['Director']= datarepo[4][i]

In [9]:
print(json.dumps((result[0]),indent=4))

{
    "Genre": "Drama",
    "Rating": "7.2",
    "Director": "Director: Sanjay Leela Bhansali",
    "Year": 2015,
    "RunTime": "An account of the romance between the Maratha general, Baji Rao I and Mastani, princess of Bundelkhand. (158 mins.)",
    "Votes": "22,156",
    "Movie Name": "Bajirao Mastani"
}


In [10]:
print(json.dumps((result),indent=4))

{
    "0": {
        "Genre": "Drama",
        "Rating": "7.2",
        "Director": "Director: Sanjay Leela Bhansali",
        "Year": 2015,
        "RunTime": "An account of the romance between the Maratha general, Baji Rao I and Mastani, princess of Bundelkhand. (158 mins.)",
        "Votes": "22,156",
        "Movie Name": "Bajirao Mastani"
    },
    "1": {
        "Genre": "Drama",
        "Rating": "8.3",
        "Director": "Director: Vikas Bahl",
        "Year": 2014,
        "RunTime": "A Delhi girl from a traditional family sets out on a solo honeymoon after her marriage gets cancelled. (146 mins.)",
        "Votes": "46,678",
        "Movie Name": "Queen"
    },
    "2": {
        "Genre": "Drama",
        "Rating": "8.3",
        "Director": "Director: Rakeysh Omprakash Mehra",
        "Year": 2013,
        "RunTime": "The truth behind the ascension of Milkha \"The Flying Sikh\" Singh who was scarred because of the India-Pakistan partition. (186 mins.)",
        "Votes": "4

In [11]:
# create dataframe
df = pd.DataFrame.from_dict(result,orient='index')
df = df[['Year', 'Movie Name', 'Rating', 'Votes','Genre','RunTime','Director']]
df.index = np.arange(1, 62)
df.head(10)

,Year,Movie Name,Rating,Votes,Genre,RunTime,Director
1,2015,Bajirao Mastani,7.2,"22,156",Drama,An account of the romance between the Maratha ...,Director: Sanjay Leela Bhansali
2,2014,Queen,8.3,"46,678",Drama,A Delhi girl from a traditional family sets ou...,Director: Vikas Bahl
3,2013,Bhaag Milkha Bhaag,8.3,"46,431",Drama,"The truth behind the ascension of Milkha ""The ...",Director: Rakeysh Omprakash Mehra
4,2012,Barfi!,8.1,"59,944",Family,Three young people learn that love can neither...,Director: Anurag Basu
5,2011,Zindagi Na Milegi Dobara,8.1,"48,927",Drama,Three friends decide to turn their fantasy vac...,Director: Zoya Akhtar
6,2010,Dabangg,6.3,"22,286",Family,A corrupt police officer faces challenges from...,Director: Abhinav Singh Kashyap
7,2009,3 Idiots,8.4,"239,938",Musical,Two friends are searching for their long lost ...,Director: Rajkumar Hirani
8,2008,Jodhaa Akbar,7.6,"22,678",Drama,A sixteenth century love story about a marriag...,Director: Ashutosh Gowariker
9,2007,Like Stars on Earth,8.5,"103,368",Family,An eight-year-old boy is thought to be a lazy ...,Director: Aamir Khan
10,2006,Rang De Basanti,8.3,"80,870",Musical,This story is about six friends who help an En...,Director: Rakesh Omprakash Mehra


In [12]:
df

,Year,Movie Name,Rating,Votes,Genre,RunTime,Director
1,2015,Bajirao Mastani,7.2,"22,156",Drama,An account of the romance between the Maratha ...,Director: Sanjay Leela Bhansali
2,2014,Queen,8.3,"46,678",Drama,A Delhi girl from a traditional family sets ou...,Director: Vikas Bahl
3,2013,Bhaag Milkha Bhaag,8.3,"46,431",Drama,"The truth behind the ascension of Milkha ""The ...",Director: Rakeysh Omprakash Mehra
4,2012,Barfi!,8.1,"59,944",Family,Three young people learn that love can neither...,Director: Anurag Basu
5,2011,Zindagi Na Milegi Dobara,8.1,"48,927",Drama,Three friends decide to turn their fantasy vac...,Director: Zoya Akhtar
6,2010,Dabangg,6.3,"22,286",Family,A corrupt police officer faces challenges from...,Director: Abhinav Singh Kashyap
7,2009,3 Idiots,8.4,"239,938",Musical,Two friends are searching for their long lost ...,Director: Rajkumar Hirani
8,2008,Jodhaa Akbar,7.6,"22,678",Drama,A sixteenth century love story about a marriag...,Director: Ashutosh Gowariker
9,2007,Like Stars on Earth,8.5,"103,368",Family,An eight-year-old boy is thought to be a lazy ...,Director: Aamir Khan
10,2006,Rang De Basanti,8.3,"80,870",Musical,This story is about six friends who help an En...,Director: Rakesh Omprakash Mehra


In [17]:
df[df['RunTime'].isnull()==True]

,Year,Movie Name,Rating,Votes,Genre,RunTime,Director


In [18]:
nans = df.shape[0] - df.dropna().shape[0]
print('%d rows have missing values' % nans)

0 rows have missing values


In [19]:
df[df['RunTime'].isnull()==True]

,Year,Movie Name,Rating,Votes,Genre,RunTime,Director


In [20]:
nans = df.shape[0] - df.dropna().shape[0]
print('%d rows have missing values' % nans)


0 rows have missing values
